In [8]:
import os
import cv2
import numpy as np
from imgaug import augmenters as iaa
from tqdm import tqdm

# Paths
dataset_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\datasets\train"
output_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\datasets\train_augmented"
os.makedirs(output_dir, exist_ok=True)

# Define augmentation operations
flip = iaa.Fliplr(1.0)
rotate1 = iaa.Affine(rotate=15)
rotate2 = iaa.Affine(rotate=-15)
crop_zoom1 = iaa.Sequential([iaa.Crop(percent=(0, 0.1)), iaa.Affine(scale=(1.05, 1.1))])
crop_zoom2 = iaa.Sequential([iaa.Crop(percent=(0, 0.1)), iaa.Affine(scale=(0.9, 0.95))])
shift1 = iaa.Affine(translate_percent={"x": 0.1, "y": 0.1})
shift2 = iaa.Affine(translate_percent={"x": -0.1, "y": -0.1})
brightness = iaa.AddToBrightness((-30, 30))
contrast = iaa.LinearContrast((0.7, 1.3))

augmentations = [flip, rotate1, rotate2, crop_zoom1, crop_zoom2, shift1, shift2, brightness, contrast]

# Count total images for global progress bar
total_images = sum([len(files) for _, _, files in os.walk(dataset_dir)])

# Global progress bar
with tqdm(total=total_images, desc="Total Progress") as pbar_total:
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        # Create output folder
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        # Loop through images
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                pbar_total.update(1)
                continue
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Save original image
            cv2.imwrite(os.path.join(output_class_path, img_name), image)

            # Apply each augmentation
            for idx, aug in enumerate(augmentations):
                aug_img = aug(image=image_rgb)
                aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug{idx+1}.jpg"
                cv2.imwrite(os.path.join(output_class_path, aug_img_name), aug_img_bgr)

            pbar_total.update(1)

print("All images augmented with 9 variations each, and originals saved!")


Total Progress: 100%|██████████| 3080/3080 [00:43<00:00, 70.09it/s]

All images augmented with 9 variations each, and originals saved!
